# Scrape Vb Google Scholar

In [1]:
import __main__ as main
import subprocess as sp
import time
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import re
notebook_name = 'scrape_vb_google_scholar'

In [2]:
vineet_google_scholar = 'https://scholar.google.com/citations?hl=en&user=zr2I_WMAAAAJ&view_op=list_works&sortby=pubdate'
sp.run(f"cd publications; wget '{vineet_google_scholar}' -O gscholar_hits.html", shell=True)

--2025-02-01 14:00:09--  https://scholar.google.com/citations?hl=en&user=zr2I_WMAAAAJ&view_op=list_works&sortby=pubdate
Resolving scholar.google.com (scholar.google.com)... 142.250.68.100
Connecting to scholar.google.com (scholar.google.com)|142.250.68.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘gscholar_hits.html’

     0K .......... .......... .......... .......... ..........  862K
    50K .......... .......... .......... .......... .......... 1.71M
   100K .......... .......... .......... .......... .......    2.55M=0.1s

2025-02-01 14:00:10 (1.38 MB/s) - ‘gscholar_hits.html’ saved [151132]



CompletedProcess(args="cd publications; wget 'https://scholar.google.com/citations?hl=en&user=zr2I_WMAAAAJ&view_op=list_works&sortby=pubdate' -O gscholar_hits.html", returncode=0)

In [4]:
with open("publications/gscholar_hits.html", 'rb') as infile:
    hits = infile.read()
hits = hits.decode('unicode-escape')

/tmp/ipykernel_35065/177708489.py:3: DeprecationWarning: invalid escape sequence '\S'
  hits = hits.decode('unicode-escape')


In [5]:
publications = re.findall("(?=<td class=\"gsc_a_t\">).*?(?=</span>)", hits)
authors_ = [re.search('(?<=<div class=\"gs_gray\">).*?(?=</div>)', x).group(0) for x in publications]
titles = [re.search('(?<=class=\"gsc_a_at\">).*?(?=</a>)', x).group(0) for x in publications]
years = [re.search('20[0-9][0-9]', x).group(0) for x in publications]
citation_links = ['https://scholar.google.com/citations?view_op=view_citation&' + re.search('(?=citation_for_view).*?(?=" )', x).group(0) for x in publications]

In [6]:
authors = []
dates = []
journals = []
links = []
for citation_link in tqdm(citation_links):
    sp.run(f"wget '{citation_link}' -O tmp.html", shell=True)
    with open("tmp.html", 'rb') as infile:
        lines = infile.read().decode('unicode-escape')
        author_list, date, journal, *_ = re.findall('(?<=gsc_oci_value">).*?(?=</div>)', lines)
        if '"' in journal:
            journal = ''
        link = re.search('(?<=class="gsc_oci_title_link" href=").*?(?=")', lines).group(0)
        authors.append(BeautifulSoup(author_list).text)
        dates.append(BeautifulSoup(date).text)
        journals.append(journal)
        links.append(link)
        
#     time.sleep(5)
    sp.run("rm tmp.html", shell=True)

  0%|                                      | 0/20 [00:00<?, ?it/s]--2025-02-01 14:00:21--  https://scholar.google.com/citations?view_op=view_citation&citation_for_view=zr2I_WMAAAAJ:4vMrXwiscB8C
Resolving scholar.google.com (scholar.google.com)... 142.250.68.100
Connecting to scholar.google.com (scholar.google.com)|142.250.68.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘tmp.html’

     0K .......... .......... .......... .......... .......... 1.25M
    50K .......... .......... .......... ...                   1.78M=0.06s

2025-02-01 14:00:22 (1.42 MB/s) - ‘tmp.html’ saved [85874]

/tmp/ipykernel_35065/4110987935.py:8: DeprecationWarning: invalid escape sequence '\S'
  lines = infile.read().decode('unicode-escape')
/tmp/ipykernel_35065/4110987935.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  dates.app

In [7]:
title_length_limit = 75
abbreviated_titles = [title if len(title) < title_length_limit else title[:title_length_limit-3]+'...' for title in titles]

In [8]:
vineet_led = [x.endswith('Bafna') and not x.startswith('Joseph Califano') for x in authors]

In [9]:
paper_list = []
for title, author_list, link, journal, year, bafna_project in zip(abbreviated_titles, authors_, links, journals, years, vineet_led):
    if not bafna_project:
        continue
    paper_list.append('\t\t\t' + f"""<li><div class=link>
\t\t\t\t<a href=\"{link}\">{title}</a>
\t\t\t\t<br>
\t\t\t\t<span>{author_list}</span>
\t\t\t\t<br>
\t\t\t\t<span>{journal}, {year}</span>
\t\t\t\t</li>
""".strip())
paper_list = '\n'.join(paper_list)

In [10]:
with open('index.html') as infile:
    lines = infile.read()
    start_papers = lines.index('<ul>') + len('<ul>\n')
    end_papers = lines.index('</ul>') - len('\n\t\t')

with open('index.html', 'w') as outfile:
    outfile.write(lines[:start_papers] + paper_list + lines[end_papers:])

In [11]:
sp.run(f"cd '{os.getcwd()}'; git add .; git commit -m 'Automated Website Update'; git push origin main", shell=True)
if not hasattr(main, '__file__'):
    sp.run(f"jupyter nbconvert --to script '{notebook_name}.ipynb'; chmod u+x {notebook_name}.py", shell=True)

Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'giprasad@MSI-7D78.(none)')
fatal: could not read Username for 'https://github.com': No such device or address
[NbConvertApp] Converting notebook scrape_vb_google_scholar.ipynb to script
[NbConvertApp] Writing 3193 bytes to scrape_vb_google_scholar.py
